In [1]:
import math
import numpy as np
import cmath as cm
import random
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import chainer
from chainer import configuration
from chainer.dataset import convert
import sklearn
from sklearn.utils import shuffle
from tabulate import tabulate

#Import the module containing the class pole
import import_ipynb
import module_poleclass
from module_poleclass import mu1, mu2, T1, T2, T3, T4, Nreal, Nimag
from module_poleclass import Ereal, Eimag, Erealfar, Eimagfar, E_exp, labelz, inspect
from module_poleclass import pole, seerealimagpart, skipduplicate, exportdata, importdata, get_traintest, directory

importing Jupyter notebook from module_poleclass.ipynb
Number of poles to be generated per class: 100000
Ndata to be generated= 3500000
Your directory is: dataset_training


In [2]:
def gen_ccdataset02(Ereal, Eimag, Erealfar, Eimagfar, inspect):
    #generate dataset for output label 02 '2 poles in [bt]'
    #define this functions as part of a module
    Emock = []
    ReT11 = []
    ImT11 = []
    labelout  = []
    data_info = []
    #Shuffle indexing
    for ndx in range(10):
        Ereal = shuffle(Ereal)
        Eimag = shuffle(Eimag)
        Erealfar = shuffle(Erealfar)
        Eimagfar = shuffle(Eimagfar)
    #Generate poles in each Riemann sheet
    for real1 in range(Nreal):
        for imag1 in range(Nimag):
            #We have to choose other poles, possibly in the same dataset
            #To avoid repeated poles, we introduce random index
            indx = skipduplicate(real1, imag1, Nreal, Nimag)
            #Possible Riemann sheets
            RSpole = [[-1,1],[-1,-1],[1,-1]]
            #Generate poles in different Riemann sheet
            pole1 = pole( RSpole[0], Ereal[indx[0][0]], Eimag[indx[1][0]] )
            pole2 = pole( RSpole[0], Ereal[indx[0][1]], Eimag[indx[1][1]] )
            pole3 = pole( RSpole[np.random.choice([0,1,2])], Erealfar[indx[0][2]], Eimagfar[indx[1][2]] )
            pole4 = pole( RSpole[np.random.choice([0,1,2])], Erealfar[indx[0][3]], Eimagfar[indx[1][3]] )
            pole5 = pole( RSpole[np.random.choice([0,1,2])], Erealfar[indx[0][4]], Eimagfar[indx[1][4]] )
            pole6 = pole( RSpole[np.random.choice([0,1,2])], Erealfar[indx[0][5]], Eimagfar[indx[1][5]] )
            #Generate energy axis
            Einput = []
            delE = (T4-T1)*197.3/(len(E_exp))
            for ndx in range(len(E_exp)):
                Einput.append(np.random.uniform(low=T1*197.3+(ndx)*delE, high=T1*197.3+(ndx+1)*delE))
            Einput = np.array(Einput)            
            #vectorize function evaluation
            k1 = np.sqrt(2.0*mu1*(Einput/197.3-T1))
            k2 = []
            #cmath can't take array input. ouch...
            for kndx in range(len(Einput)):
                k2.append(cm.sqrt(2.0*mu2*(Einput[kndx]/197.3-T2)))
            k2 = np.asarray(k2)
            #calculate S-matrix contribution of each pole
            smat1 = pole1.smat(k1,k2)
            smat2 = pole2.smat(k1,k2)
            smat3 = pole3.smat(k1,k2)
            smat4 = pole4.smat(k1,k2)
            smat5 = pole5.smat(k1,k2)
            smat6 = pole6.smat(k1,k2)
            #random switch for irrelevant poles, set q=1 if the pole is relevant
            q1 = 1
            q2 = 1
            q3 = np.random.choice([0,1])
            q4 = np.random.choice([0,1])
            q5 = np.random.choice([0,1])
            q6 = np.random.choice([0,1])
            #calculate combined S-matrix
            SMAT = ((1-q1)+q1*smat1)*((1-q2)+q2*smat2)*((1-q3)+q3*smat3)*((1-q4)+q4*smat4)*((1-q5)+q5*smat5)*((1-q6)+q6*smat6)
            #get partial-wave amplitude
            PWA = (SMAT - 1.0)/(2*(1j))
            #collect input data
            Emock.append(Einput)
            ReT11.append(PWA.real)
            ImT11.append(PWA.imag)                    
            labelout.append(2)
            #collect information about the poles and their shadow poles
            if inspect == True:
                information = [[pole1.pos, pole2.pos, pole3.pos, pole4.pos, pole5.pos, pole6.pos],
                               [pole1.RS, pole2.RS, pole3.RS, pole4.RS, pole5.RS, pole6.RS],
                               [pole1.shadow, pole2.shadow, pole3.shadow, pole4.shadow, pole5.shadow, pole6.shadow],
                               [q1, q2, q3, q4, q5, q6]
                              ]
                data_info.append(information)            
    return Emock, ReT11, ImT11, labelout, data_info          

In [3]:
Einput02, ReT1102, ImT1102, labelout02, data_info02  = gen_ccdataset02(Ereal, Eimag, Erealfar, Eimagfar, inspect)
out = directory
pickle.dump (Einput02, open(os.path.join(out,'Einput02.pkl'),'wb'), protocol=4)
pickle.dump (ReT1102, open(os.path.join(out,'ReT1102.pkl'),'wb'), protocol=4)
pickle.dump (ImT1102, open(os.path.join(out,'ImT1102.pkl'),'wb'), protocol=4)
pickle.dump (labelout02, open(os.path.join(out,'labelout02.pkl'),'wb'), protocol=4)
pickle.dump (data_info02, open(os.path.join(out,'data_info02.pkl'),'wb'), protocol=4)
#The following will collect the data that will go to the input layer:
#You can design a DNN with Einput and T11sqr in the input layer
T11sqr02 = [x**2.0 + y**2.0 for x, y in zip(ReT1102, ImT1102)]
T11sqr02 = np.concatenate((Einput02,T11sqr02), axis=1)
#or you can design a DNN with Einput, ReT11 and ImT11 in the input layer
T1102 = np.concatenate((Einput02,ReT1102, ImT1102), axis=1)
pickle.dump (T11sqr02, open(os.path.join(out,'T11sqr02.pkl'),'wb'), protocol=4)
pickle.dump (T1102, open(os.path.join(out,'T1102.pkl'),'wb'), protocol=4)
del Einput02, ReT1102, ImT1102, labelout02, data_info02, T11sqr02, T1102